In [ ]:
import os
import PIL
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [ ]:
#---Edit Between Lines---
directory = 'DirtyLines'
#------------------------
authors = os.listdir(directory)

# Here we can see that our how many files our Datafolder have which is 13353 for IAM

In [ ]:
TotalFiles = 0 
for author in authors : 
    forms = os.listdir(os.path.join(directory,author))
    for form in forms :
        files = os.listdir(os.path.join(directory,author,form))
        TotalFiles += len(files)
print(TotalFiles)

# Now we are going to clear these 13353 images based on our selections;
# Width > 1620
# Height < 174 && imsize > 68

In [ ]:
EleminateCount = 0
for author in authors : 
    forms = os.listdir(os.path.join(directory,author))
    for form in forms :
        files = os.listdir(os.path.join(directory,author,form))
        for file in files :
            imsize = 0
            with Image.open(os.path.join(directory,author,form,file)) as test_image:
                imsize = test_image.size
            if imsize[0] <1620 or imsize[1] >174 or imsize[1] < 68 :
                try:
                    EleminateCount += 1
                    os.remove(os.path.join(directory,author,form,file))
                except Exception as e:
                    EleminateCount -= 1
                    print("Remove failed" , os.path.join(directory,author,form,file) , " , " , e)
print(EleminateCount)

# Now we are going to change file orientation as we are going to use as Author-> Files.

In [ ]:
f = open("forms.txt", "r")
filetxt = f.read().split("\n")
fileAuthorDict = dict()
for line in filetxt[:-1] :
    splitedline = line.split(" ")
    fileAuthorDict[str(splitedline[0])] = str(splitedline[1])

len(fileAuthorDict)

In [ ]:
FromDirectory = directory
#---Edit Between Lines---
ToDirectory = ".\\CleanedAllTriplets"
#------------------------
authors = os.listdir(FromDirectory)

In [ ]:
import shutil
Copycounter = 0 
for author in authors : 
    forms = os.listdir(os.path.join(directory,author))
    for form in forms :
        try:
            os.mkdir(os.path.join(ToDirectory,fileAuthorDict.get(form)))
        except Exception as e :
            pass
        files = os.listdir(os.path.join(directory,author,form))
        for file in files:
            Copycounter += 1
            shutil.copy(os.path.join(directory,author,form,file), os.path.join(ToDirectory,fileAuthorDict.get(form)))
            
Copycounter

In [ ]:
Top100 = dict()
authors = os.listdir(ToDirectory)
for author in authors:
    files = os.listdir(os.path.join(ToDirectory,author))
    Top100[author] = len(files)

sort_orders = sorted(Top100.items(), key=lambda x: x[1], reverse=True)
print(sort_orders[:100])

In [ ]:
#---Edit Between Lines---
cleanDirectory=".\\CleanTop100Data"
#------------------------
import shutil
for item in sort_orders[:100]:
    shutil.copytree(os.path.join(ToDirectory,item[0]), os.path.join(cleanDirectory,item[0]))

In [ ]:
FileCounter = 0
for item in sort_orders[:100]:
    FileCounter += item[1]
FileCounter

# Now here , we have our Top100CleanedData ready for preprocessing . our clean Directorys name is CleanTop100Data.

In [ ]:
from AuthorsUtility import *
from skimage import io
#padheight => Only Pads HEIGHT
#pad => 2270 - 200 padding !!! FORCED BECAUSE TENSOR WANTS EQUALIZED SIZES !!! 

In [ ]:
#---Edit Between Lines---
ToCleanDirectory = ".\\HeightPaddedTop100Data"
#------------------------
for author in os.listdir(os.path.join(cleanDirectory)):
    try:
        os.mkdir(os.path.join(ToCleanDirectory,author))
    except:
        pass

In [ ]:
for author in os.listdir(os.path.join(cleanDirectory)):
    for images in os.listdir(os.path.join(cleanDirectory,author)):
        file = io.imread(os.path.join(cleanDirectory,author,images))
        io.imsave(os.path.join(ToCleanDirectory,author)+os.path.sep+images,pad(file))

# Now that we have our HEIGHT padded files we can apply adaptive treshold on them based on IAM binarize values

In [ ]:
Thresholder = {}
with open('lines.txt') as f:
    lines = f.readlines()
    for line in lines : 
        properties =line.split(' ')
        Thresholder[properties[0]] = int(properties[2])

In [ ]:
Thresholder

In [ ]:
#---Edit Between Lines---
ToCleanDirectory = ".\\HeightPaddedTresholdedTop100Data"
#------------------------

In [ ]:
for author in os.listdir(os.path.join(cleanDirectory)):
    try:
        os.mkdir(os.path.join(ToCleanDirectory,author))
    except:
        pass

In [ ]:
#---Edit Between Lines---
PaddedDirectory = ".\\HeightPaddedTop100Data"
#------------------------
for author in os.listdir(os.path.join(PaddedDirectory)):
    for images in os.listdir(os.path.join(PaddedDirectory,author)):
        file = io.imread(os.path.join(PaddedDirectory,author,images))
        io.imsave(os.path.join(ToCleanDirectory,author)+os.path.sep+images,threshold(file,Thresholder.get(images.split('.')[0])))

# Now That we have everything ready . We can split our dataset or directly genereate PT file from our dataset . Here i chose to generate directly but at the bottom i will share splitting method too.

In [ ]:
import os
import torch
import glob
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import math
import random
import unittest

In [ ]:
DatasetPath = "HeightPaddedTresholdedTop100Data"

In [ ]:
ReadTupleList = []
for authors in os.listdir(os.path.join(DatasetPath)):
    for images in os.listdir(os.path.join(DatasetPath,authors)):
        ReadTupleList.append((os.path.join(DatasetPath,authors,images),authors))

In [ ]:
ReadTupleList

In [ ]:
len(ReadTupleList)

In [ ]:
random.shuffle(ReadTupleList)

In [ ]:
ReadTupleList

In [ ]:
ImageTensor = []
LabelTensor = []
for file , label in ReadTupleList :
    img = io.imread(file)
    ImageTensor.append(img)
    LabelTensor.append(int(label))

In [ ]:
ImageTensor

In [ ]:
LabelTensor

In [ ]:
BigTensor = (torch.ByteTensor(ImageTensor).view(-1, 200, 2270),torch.LongTensor(LabelTensor))

In [ ]:
with open('Dataset.pt', 'wb') as f:
            torch.save(BigTensor, f)

# Do NOT RUN below if not needed ! ...

# Dataset Splitting Code !! 

In [ ]:
import os
import random
import shutil
FolderDirectory = '.\\PaddedTresholdedTop100Triplet'
for author in os.listdir(os.path.join(FolderDirectory)):
    ImageCount = len(os.listdir(os.path.join(FolderDirectory,author)))
    selectlist = list(map(lambda _: random.choice(os.listdir(os.path.join(FolderDirectory,author))), range(int(ImageCount * 3 / 27))))
    for images in os.listdir(os.path.join(FolderDirectory,author)):
        if images in selectlist : 
            shutil.copy(os.path.join(FolderDirectory,author,images), os.path.join('Top100Testing',author,images))
        else : 
            shutil.copy(os.path.join(FolderDirectory,author,images), os.path.join('Top100Training',author,images))
